In [5]:
import random
import collections

# Pennylane
import pennylane as qml
from pennylane import numpy as np
 
# Tensorflow
import tensorflow as tf

In [6]:
def reduce_image(x):
    x = tf.reshape(x, [1, 28, 28, 1])
    x = tf.image.resize(x, [4, 4])
    x = tf.reshape(x, [4, 4])
    x = x / 255
    return x.numpy()

def remove_contradicting(xs, ys):
    mapping = collections.defaultdict(set)
    for x, y in zip(xs, ys):
        mapping[str(x)].add(y)

    return zip(*((x, y) for x, y in zip(xs, ys) if len(mapping[str(x)]) == 1))

#Simple implementation of encoding image in a Pennylane's circuit
def convert_to_circuit(image):
    
    values = np.ndarray.flatten(image)
    for i, value in enumerate(values):
        if value > 0.5:
            qml.PauliX(wires = i)

In [7]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

print("Number of original training examples:", len(x_train))
print("Number of original test examples:", len(x_train))

x_train, y_train = zip(*((x, y) for x, y in zip(x_train, y_train) if y in [3, 6]))
x_test, y_test = zip(*((x, y) for x, y in zip(x_test, y_test) if y in [3, 6]))

x_train = [reduce_image(x) for x in x_train]
x_test = [reduce_image(x) for x in x_test]

x_train, y_train = remove_contradicting(x_train, y_train)
x_test, y_test = remove_contradicting(x_test, y_test)

print("Number of filtered training examples:", len(x_train))
print("Number of filtered test examples:", len(x_test))

Number of original training examples: 60000
Number of original test examples: 60000
Number of filtered training examples: 11520
Number of filtered test examples: 1906


In [ ]:
# Maybe is better here to start implementing Complete-depth learning using Pytorch interface.